# Export PyTorch Model to ONNX Format

This notebook demonstrates how to export a pre-trained PyTorch image classification model to ONNX format for deployment with OpenVINO Model Server.

We use **SqueezeNet 1.1**, a lightweight convolutional neural network designed for efficient inference on resource-constrained environments.

## 1. Import Required Libraries

Import PyTorch and torchvision for model loading and ONNX export.

In [ ]:
import torch
import torchvision.models as models
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

## 2. Load Pre-trained Model

Load SqueezeNet 1.1 with weights pre-trained on ImageNet.

**Model Details:**
- Architecture: SqueezeNet 1.1
- Input size: 224x224 RGB images
- Output: 1000 ImageNet classes
- Model size: ~5 MB

In [ ]:
# Load pre-trained SqueezeNet 1.1 model
model = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.IMAGENET1K_V1)

# Set model to evaluation mode (disables dropout, batch norm in training mode)
model.eval()

print("Model loaded successfully")
print(f"Model architecture: SqueezeNet 1.1")

## 3. Create Output Directory

Ensure the output directory exists for saving the ONNX model.

In [ ]:
# Create output directory
output_dir = Path("models/openvino")
output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / "model.onnx"
print(f"Output path: {output_path}")

## 4. Export Model to ONNX Format

Create a dummy input tensor matching the model's expected input shape and export to ONNX.

**ONNX Export Parameters:**
- `input_names`: Names for input tensors
- `output_names`: Names for output tensors
- `dynamic_axes`: Allow dynamic batch size for inference
- `opset_version`: ONNX opset version (13 is compatible with OpenVINO)

In [ ]:
# Create dummy input tensor (batch_size=1, channels=3, height=224, width=224)
dummy_input = torch.randn(1, 3, 224, 224)

# Export to ONNX format
torch.onnx.export(
    model,                          # Model to export
    dummy_input,                    # Example input tensor
    str(output_path),               # Output file path
    input_names=["input"],          # Input tensor name
    output_names=["output"],        # Output tensor name
    dynamic_axes={
        "input": {0: "batch_size"},  # Allow variable batch size
        "output": {0: "batch_size"}
    },
    opset_version=13,               # ONNX opset version
    export_params=True,             # Export trained parameters
    do_constant_folding=True        # Optimize constant folding
)

print(f"\nModel exported successfully to: {output_path}")

## 5. Verify ONNX Export

Check that the ONNX file was created and display its size.

In [ ]:
# Verify file exists and get size
if output_path.exists():
    file_size = output_path.stat().st_size
    file_size_mb = file_size / (1024 * 1024)
    print(f"✓ ONNX model created successfully")
    print(f"  File: {output_path}")
    print(f"  Size: {file_size_mb:.1f} MB")
else:
    print("✗ Error: ONNX file was not created")

## Summary

You have successfully:
1. Loaded a pre-trained SqueezeNet 1.1 model from torchvision
2. Exported the model to ONNX format with dynamic batch size support
3. Saved the ONNX model to `models/openvino/model.onnx`

The exported model is ready to be uploaded to S3 object storage and deployed using OpenVINO Model Server.